In [0]:
#Đọc file dữ liệu
df = "dbfs:/FileStore/shared_uploads/danghia2002@gmail.com/diemthi2020.csv"

In [0]:
#Đọc tệp thành RDD
rdd = sc.textFile(df)

In [0]:
#Nạp dữ liệu
def fsplit(s):
    l = s.split(',')
    d = {'TT': int(l[0]), 'SBD': l[11]}
    if l[1] != '':
        d['Dia'] = float(l[1])
    if l[2] != '':
        d['GDCD'] = float(l[2])
    if l[3] != '':
        d['Hoa'] = float(l[3])
    if l[4] != '':
        d['Li'] = float(l[4])
    if l[6] != '':
        d['NN'] = float(l[6])
    if l[7] != '':
        d['Sinh'] = float(l[7])
    if l[8] != '':
        d['Su'] = float(l[8])
    if l[9] != '':
        d['Toan'] = float(l[9])
    if l[10] != '':
        d['Van'] = float(l[10])
    return [d]

rdd = sc.textFile('dbfs:/FileStore/shared_uploads/danghia2002@gmail.com/diemthi2020.csv').filter(lambda s: s[0] != 'T').flatMap(fsplit)

In [0]:
# 1. Đếm số bài thi điểm 0 từng môn.
rdd1 = rdd.flatMap(lambda h: [(x, y) for x, y in h.items()]) \
.filter(lambda t: not (t[0] in ['TT', 'SBD'])) \
.groupByKey() \
.mapValues(lambda values: sum(1 for v in values if v == 0))

# In ra danh sách số bài thi điểm 0 từng môn
result = rdd1.collect()
for subject, count in result:
    print(f"Số bài thi điểm 0 môn {subject}: {count}")

Số bài thi điểm 0 môn GDCD: 42
Số bài thi điểm 0 môn Toan: 7
Số bài thi điểm 0 môn Hoa: 13
Số bài thi điểm 0 môn Sinh: 23
Số bài thi điểm 0 môn Dia: 125
Số bài thi điểm 0 môn NN: 18
Số bài thi điểm 0 môn Su: 18
Số bài thi điểm 0 môn Van: 32
Số bài thi điểm 0 môn Li: 8


In [0]:
# Tìm môn thi có nhiều bài thi đạt điểm 10 nhất và số bài thi điểm 10 của môn đó
rdd1 = rdd.flatMap(lambda h: [(x, y) for x, y in h.items()]) \
.filter(lambda t: not (t[0] in ['TT', 'SBD'])) \
.filter(lambda t: t[1] == 10) \
.map(lambda t: (t[0], 1)) \
.reduceByKey(lambda a, b: a + b) \
.sortBy(lambda t: t[1], ascending=False)

# Lấy thông tin môn thi có nhiều bài thi đạt điểm 10 nhất
top_subject = rdd1.first()
subject = top_subject[0]
count = top_subject[1]

# In ra kết quả
print(f"Môn thi có nhiều bài thi đạt điểm 10 nhất là {subject} với {count} bài thi điểm 10.")

Môn thi có nhiều bài thi đạt điểm 10 nhất là GDCD với 4195 bài thi điểm 10.


In [0]:
rdd1 = rdd.filter(lambda s: s[0] != 'T').map(fsplit)
def average_score(student):
    total_score = sum(value for key, value in student.items() if key not in ['TT', 'SBD'])
    total_subjects = len(student) - 2 
    return total_score / total_subjects
top_scoring_student = rdd.max(key=average_score)
print(f"Sinh viên có điểm trung bình cao nhất có số báo danh: {top_scoring_student['SBD']} và điểm trung bình: {average_score(top_scoring_student)}")

Sinh viên có điểm trung bình cao nhất có số báo danh: 02065027 và điểm trung bình: 9.783333333333333


In [0]:
# Tạo RDD chứa số môn thi của từng thí sinh
rdd_mon_thi = rdd.map(lambda student: sum(1 for key in student.keys() if key not in ['TT', 'SBD']))

# Đếm số lượng thí sinh cho từng số môn thi
thi_sinh_theo_mon_thi = rdd_mon_thi.countByValue()

# In kết quả
for so_mon_thi, so_thi_sinh in sorted(thi_sinh_theo_mon_thi.items()):
    print(f"{so_mon_thi} môn: {so_thi_sinh} thí sinh")

1 môn: 323 thí sinh
2 môn: 589 thí sinh
3 môn: 29024 thí sinh
4 môn: 71432 thí sinh
5 môn: 7935 thí sinh
6 môn: 761182 thí sinh
9 môn: 1 thí sinh


In [0]:
#Tính điểm trung bình bài thi Tự nhiên, bài thi Xã hội. Biết rằng điểm bài thi tổ hợp là tổng điểm thi 3 môn thành phần của bài thi đó.
diem_to_hop = rdd.flatMap(lambda h: [(x,y) for x, y in h.items()])
rdd1 = diem_to_hop.filter(lambda t: t[0] in ['GDCD', 'Dia', 'Su']).map(lambda x: x[1])
tbxh = rdd1.mean()
rdd2 = diem_to_hop.filter(lambda t: t[0] in ['Li', 'Hoa', 'Sinh']).map(lambda x: x[1])
tbtn = rdd2.mean()
print("Trung bình tổ hợp xã hội:",round(tbxh,2))
print("Trung bình tổ hợp tự nhiên:", round(tbtn,2))

Trung bình tổ hợp xã hội: 6.62
Trung bình tổ hợp tự nhiên: 6.34


In [0]:
# Tính điểm bài thi tổ hợp Tự nhiên (Hoa, Li, Sinh)
rdd_tn = rdd.map(lambda x: x.get('Hoa', 0) + x.get('Li', 0) + x.get('Sinh', 0))
rdd_tn = rdd_tn.filter(lambda x: x != 0)
trung_binh_tn = rdd_tn.mean()

# Tính điểm bài thi tổ hợp Xã hội (GDCD, Địa, Sử)
rdd_xh = rdd.map(lambda x: x.get('GDCD', 0) + x.get('Dia', 0) + x.get('Su', 0))
rdd_xh = rdd_xh.filter(lambda x: x != 0)
trung_binh_xh = rdd_xh.mean()

print("Trung bình điểm bài thi Tự nhiên:", round(trung_binh_tn,2))
print("Trung bình điểm bài thi Xã hội:", round(trung_binh_xh,2))

Trung bình điểm bài thi Tự nhiên: 18.73
Trung bình điểm bài thi Xã hội: 18.69


In [0]:
#Vẽ biểu đồ biểu diễn số lượng bài thi Tự nhiên và Xã hội theo từng mứ điểm


---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-1247489471686340>:5
      2 import matplotlib.pyplot as plt
      4 # Tạo DataFrame chứa số lượng bài thi Tự nhiên và Xã hội theo từng điểm
----> 5 result_df = df.filter((col('mon_thi') == 'Tu nhien') | (col('mon_thi') == 'Xa hoi')).groupBy('mon_thi', 'diem').agg({'so_bao_danh': 'count'}).toPandas()
      7 # Vẽ biểu đồ
      8 plt.figure(figsize=(10, 6))

AttributeError: 'str' object has no attribute 'filter'